In [2]:
import pandas as pd
import numpy as np

TIME_THRESHOLD = 4*10**7
TIME_STEP = 9*10**7

In [3]:
## parse into data frames, remove unnecessary cols, normalize time

files = ["chocolate-bag-_epuck_10_odometry_filtered.csv", "chocolate-bag-_epuck_13_odometry_filtered.csv",
         "chocolate-bag-_epuck_16_odometry_filtered.csv", "chocolate-bag-_epuck_23_odometry_filtered.csv",
         "chocolate-bag-_epuck_24_odometry_filtered.csv", "chocolate-bag-_epuck_25_odometry_filtered.csv",
         "chocolate-bag-_epuck_26_odometry_filtered.csv", "chocolate-bag-_epuck_31_odometry_filtered.csv",
         "chocolate-bag-_epuck_33_odometry_filtered.csv", "chocolate-bag-_epuck_37_odometry_filtered.csv",
         "chocolate-bag-_epuck_38_odometry_filtered.csv", "chocolate-bag-_epuck_3_odometry_filtered.csv",
         "chocolate-bag-_epuck_40_odometry_filtered.csv", "chocolate-bag-_epuck_43_odometry_filtered.csv",
         "chocolate-bag-_epuck_45_odometry_filtered.csv"]

def get_clean_df(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df.sort_values(by=['field.header.seq'])

    result = pd.DataFrame()

    result["time"] = df["%time"]

    result["pos_x"] = df["field.pose.pose.position.x"]
    result["pos_y"] = df["field.pose.pose.position.y"]
    # result["pos_z"] = df["field.pose.pose.position.z"]

    # result["ori_x"] = df["field.pose.pose.orientation.x"]
    # result["ori_y"] = df["field.pose.pose.orientation.y"]
    # result["ori_z"] = df["field.pose.pose.orientation.z"]
    # result["ori_w"] = df["field.pose.pose.orientation.w"]

    return result

start_times = []
dfs = []
for file in files:
    df = get_clean_df(file)
    start_times.append(df["time"].iloc[0])
    dfs.append(df)
start_time = min(start_times)

result = []
for df in dfs:
    df["time"] = df["time"] - start_time
    result.append(df)
    print(df.head())
dfs = result


        time     pos_x     pos_y
0    2677241 -0.831550 -0.276214
1   35905302 -0.831777 -0.276176
2   69370218 -0.832233 -0.275590
3  102703908 -0.834115 -0.273352
4  135910218 -0.838907 -0.270118
        time     pos_x     pos_y
0   15007712 -0.024244  0.379216
1   48474832 -0.024287  0.380983
2   81594750 -0.024998  0.382775
3  115072039 -0.025728  0.383993
4  148351256 -0.026785  0.385805
        time     pos_x     pos_y
0   28858991 -0.861442  0.054936
1   61977897 -0.861667  0.055123
2   95971957 -0.862344  0.055240
3  128492929 -0.862670  0.054643
4  162426326 -0.863125  0.054712
        time     pos_x     pos_y
0   25372790  0.027821 -0.122642
1   58673517  0.026260 -0.121835
2   92236828  0.024389 -0.121274
3  125388295  0.023773 -0.121792
4  159651481  0.022863 -0.122055
        time     pos_x     pos_y
0   14878740 -0.560089  0.125161
1   48369134 -0.559991  0.125050
2   81667126 -0.562298  0.125617
3  114921757 -0.566438  0.126156
4  148317122 -0.571274  0.127090
        ti

In [35]:
## get end_time

end_times = [] 
for df in dfs:
    end_times.append(df["time"].iloc[-1])
end_time = max(end_times) 
print(end_time)

119353780933


In [36]:
## note time is in nano seconds 
## this takes a while the code is not that efficient (this code took on my machine 26 min to execute 

def discretize_time_series(df: pd.DataFrame) -> pd.DataFrame:
    temp = pd.DataFrame(columns = df.columns)
    temp.loc[0] = np.nan
    nan_row = temp.loc[0]

    result = []
    result.append(df.loc[0])
    for i in range(int(end_time / TIME_STEP)):    
        time_step = i * TIME_STEP
        mask = (df['time'] > time_step - TIME_THRESHOLD ) & (df['time'] <= time_step + TIME_THRESHOLD)
        if len(mask) > 0:
            result.append(df.loc[mask].mean())
        else:
            result.append(nan_row)

    return pd.DataFrame.from_records(result)

result = []
for df in dfs:
    temp = discretize_time_series(df)
    temp = temp.drop(columns=["time"])
    temp = temp.interpolate(method='linear', limit_direction='forward')
    result.append(temp)

dfs = result

In [37]:
original = dfs

In [38]:
dfs = original


In [39]:

for i in range(len(dfs)):
    dfs[i] = dfs[i].rename(columns={"pos_x": "bot{i}_x".format(i=i), "pos_y": "bot{i}_y".format(i=i)})

result = pd.concat(dfs, axis=1, join="inner")
result.head()

,bot0_x,bot0_y,bot1_x,bot1_y,bot2_x,bot2_y,bot3_x,bot3_y,bot4_x,bot4_y,...,bot10_x,bot10_y,bot11_x,bot11_y,bot12_x,bot12_y,bot13_x,bot13_y,bot14_x,bot14_y
0,-0.831550,-0.276214,-0.024244,0.379216,-0.861442,0.054936,0.027821,-0.122642,-0.560089,0.125161,...,-0.966687,-0.215291,-0.511696,-0.128674,-0.719000,0.050655,-0.280301,-0.155063,-0.592001,-0.590158
1,-0.831663,-0.276195,-0.024244,0.379216,-0.861442,0.054936,0.027821,-0.122642,-0.560089,0.125161,...,-0.966687,-0.215291,-0.511696,-0.128674,-0.720961,0.050508,-0.280301,-0.155063,-0.592001,-0.590158
2,-0.833174,-0.274471,-0.025363,0.383384,-0.862227,0.055002,0.024807,-0.121634,-0.564368,0.125886,...,-0.961311,-0.209174,-0.522836,-0.126167,-0.722922,0.050360,-0.277407,-0.147897,-0.592281,-0.590046
3,-0.840034,-0.267858,-0.027649,0.387092,-0.863391,0.054935,0.022327,-0.121658,-0.574389,0.127736,...,-0.949492,-0.202159,-0.534908,-0.123557,-0.731137,0.051397,-0.273180,-0.138943,-0.581423,-0.577479
4,-0.839808,-0.267115,-0.028163,0.393622,-0.866709,0.055450,0.019971,-0.123094,-0.590011,0.130054,...,-0.935467,-0.193064,-0.543349,-0.121482,-0.741856,0.054506,-0.272331,-0.135424,-0.566194,-0.565386


In [40]:
result.to_csv("all_bot_pos.csv", index=False)